In [109]:
import pandas as pd 
import numpy as np 
from geopy.distance import geodesic 

In [110]:
streetlights_df = pd.read_csv("/Users/stevenqie/Downloads/Streetlights_with_block.csv")
locations_df = pd.read_csv("/Users/stevenqie/Downloads/Location.csv")
apartments_df = pd.read_csv("/Users/stevenqie/Downloads/Apartments_with_block.csv")
crashes_df = pd.read_csv("/Users/stevenqie/Downloads/Crash_with_block.csv")

In [111]:
def calculate_distance(lat1, lon1, lat2, lon2):
    return geodesic((lat1, lon1), (lat2, lon2)).meters

proximity_radius = 500 #in meters 

In [112]:
apartments_df.head()
len(apartments_df)

2585

In [113]:
streetlights_df.head()

,streetlight_id,pole_material,wattage,height,color,latitude,longitude,block
0,1,Steel,250.0,6.0,Black,40.146307,-88.258109,77
1,2,Aluminum,NaN,4.0,Black,40.115463,-88.239671,48
2,3,Cast Aluminum,85.0,1.0,Black,40.116243,-88.238545,49
3,6,Aluminum,250.0,6.0,Black,40.116252,-88.236536,49
4,7,Cast Iron,55.0,7.0,Green,40.119176,-88.260596,56


In [114]:
# create a new column to store the number of streetlights within the proximity radius
apartments_df['num_streetlights'] = 0 
# create a new colum to store the average wattage of those streetlights within the proximity radius
apartments_df['avg_wattage'] = 0

#loop through each apartment 
for index1, apartment in apartments_df.iterrows():
    print(index1)
    #grab its lat and long 
    apartment_location = (apartment["latitude"], apartment["longitude"])
    count = 0 
    wattage_sum = 0 

    #loop through each streetlight and calculate distance betweeen aparmtent and streetlight 
    for index2, streetlight in streetlights_df.iterrows():
        streetlight_location = (streetlight["latitude"], streetlight["longitude"])
        distance = calculate_distance(apartment_location[0], apartment_location[1], streetlight_location[0], streetlight_location[1])
        #check if distance is less than proximity radius
        if distance <= proximity_radius:
            #add to count
            count += 1 
            wattage_sum += streetlight["wattage"] if not np.isnan(streetlight["wattage"]) else 0
    apartments_df.loc[index1, 'num_streetlights'] = count
    apartments_df.loc[index1, 'avg_wattage'] = wattage_sum / count if count > 0 else 0
    print(f"Apartment has {apartments_df.loc[index1, 'num_streetlights']} streetlights within {proximity_radius} meters")
    print(f"Average wattage of streetlights around this apartment is {apartments_df.loc[index1, 'avg_wattage']} watts")



0


/var/folders/9y/szyfh1mj3gg6w5909wl0dz6r0000gn/T/ipykernel_65248/820063072.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '31.76' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  apartments_df.loc[index1, 'avg_wattage'] = wattage_sum / count if count > 0 else 0


Apartment has 25 streetlights within 500 meters
Average wattage of streetlights around this apartment is 31.76 watts
1
Apartment has 26 streetlights within 500 meters
Average wattage of streetlights around this apartment is 30.53846153846154 watts
2
Apartment has 27 streetlights within 500 meters
Average wattage of streetlights around this apartment is 33.111111111111114 watts
3
Apartment has 29 streetlights within 500 meters
Average wattage of streetlights around this apartment is 48.06896551724138 watts
4
Apartment has 23 streetlights within 500 meters
Average wattage of streetlights around this apartment is 34.608695652173914 watts
5
Apartment has 37 streetlights within 500 meters
Average wattage of streetlights around this apartment is 129.56756756756758 watts
6
Apartment has 37 streetlights within 500 meters
Average wattage of streetlights around this apartment is 129.56756756756758 watts
7
Apartment has 39 streetlights within 500 meters
Average wattage of streetlights around this

In [115]:
apartments_df = apartments_df.iloc[:-1]
apartments_df

,address,safestay_score,latitude,longitude,block,num_streetlights,avg_wattage
0,2009 Moreland Blvd,5,40.140570,-88.253256,77,25,31.760000
1,2011 Moreland Blvd,6,40.140652,-88.252074,77,26,30.538462
2,2007 Moreland Blvd,7,40.140169,-88.252625,77,27,33.111111
3,2003 Moreland Blvd,8,40.139696,-88.252056,77,29,48.068966
4,2005 Moreland Blvd,0,40.139423,-88.253174,67,23,34.608696
...,...,...,...,...,...,...,...
2579,2407 Leeper Dr,0,40.082481,-88.279925,15,4,0.000000
2580,2507 Leeper Dr,9,40.081463,-88.279758,15,4,0.000000
2581,2401-2415 Morrissey Park Dr,5,40.083264,-88.266737,16,4,0.000000
2582,2417-2427 Morrissey Park Dr,8,40.082793,-88.266738,16,4,0.000000


In [116]:
crashes_df.head()

,crash_id,crash_severity,traffic_control,year,road_surface,latitude,longitude,block
0,1,A-Injury,No Controls,1999,Dry,40.137604,-88.271059,65
1,2,A-Injury,Traffic signal,1999,Dry,40.118258,-88.257847,57
2,3,A-Injury,No Controls,1999,Dry,40.126979,-88.199642,59
3,4,A-Injury,Traffic signal,2000,Dry,40.308221,-88.151385,99
4,5,A-Injury,No Controls,2000,Dry,40.117704,-88.204235,49


In [117]:
copy = apartments_df.copy()
copy 

,address,safestay_score,latitude,longitude,block,num_streetlights,avg_wattage
0,2009 Moreland Blvd,5,40.140570,-88.253256,77,25,31.760000
1,2011 Moreland Blvd,6,40.140652,-88.252074,77,26,30.538462
2,2007 Moreland Blvd,7,40.140169,-88.252625,77,27,33.111111
3,2003 Moreland Blvd,8,40.139696,-88.252056,77,29,48.068966
4,2005 Moreland Blvd,0,40.139423,-88.253174,67,23,34.608696
...,...,...,...,...,...,...,...
2579,2407 Leeper Dr,0,40.082481,-88.279925,15,4,0.000000
2580,2507 Leeper Dr,9,40.081463,-88.279758,15,4,0.000000
2581,2401-2415 Morrissey Park Dr,5,40.083264,-88.266737,16,4,0.000000
2582,2417-2427 Morrissey Park Dr,8,40.082793,-88.266738,16,4,0.000000


In [144]:
#get crashes per block using group by 
crashes_df = pd.read_csv("/Users/stevenqie/Downloads/Crash_with_block.csv")
crashes_df = crashes_df.groupby("block").size().reset_index(name='num_crashes')
crashes_df = crashes_df[crashes_df['block'] >= 0]
crashes_df = crashes_df.reset_index(drop=True)
crashes_df


,block,num_crashes
0,8,1
1,9,2
2,13,3
3,17,1
4,18,2
5,19,6
6,23,4
7,24,1
8,25,2
9,27,3


In [145]:
newone = pd.merge(copy, crashes_df, on='block', how='left')
newone = newone.fillna(0)
newone

,address,safestay_score,latitude,longitude,block,num_streetlights,avg_wattage,num_crashes
0,2009 Moreland Blvd,5,40.140570,-88.253256,77,25,31.760000,6.0
1,2011 Moreland Blvd,6,40.140652,-88.252074,77,26,30.538462,6.0
2,2007 Moreland Blvd,7,40.140169,-88.252625,77,27,33.111111,6.0
3,2003 Moreland Blvd,8,40.139696,-88.252056,77,29,48.068966,6.0
4,2005 Moreland Blvd,0,40.139423,-88.253174,67,23,34.608696,9.0
...,...,...,...,...,...,...,...,...
2579,2407 Leeper Dr,0,40.082481,-88.279925,15,4,0.000000,0.0
2580,2507 Leeper Dr,9,40.081463,-88.279758,15,4,0.000000,0.0
2581,2401-2415 Morrissey Park Dr,5,40.083264,-88.266737,16,4,0.000000,0.0
2582,2417-2427 Morrissey Park Dr,8,40.082793,-88.266738,16,4,0.000000,0.0


In [146]:
#merge with locations to get whether or not an apartment has a police station in it's block 
extranewone = newone.merge(locations_df, on='block', how='left')
extranewone = extranewone.fillna(0)
extranewone

,address,safestay_score,latitude,longitude,block,num_streetlights,avg_wattage,num_crashes,has_police_station
0,2009 Moreland Blvd,5,40.140570,-88.253256,77,25,31.760000,6.0,1
1,2011 Moreland Blvd,6,40.140652,-88.252074,77,26,30.538462,6.0,1
2,2007 Moreland Blvd,7,40.140169,-88.252625,77,27,33.111111,6.0,1
3,2003 Moreland Blvd,8,40.139696,-88.252056,77,29,48.068966,6.0,1
4,2005 Moreland Blvd,0,40.139423,-88.253174,67,23,34.608696,9.0,1
...,...,...,...,...,...,...,...,...,...
2579,2407 Leeper Dr,0,40.082481,-88.279925,15,4,0.000000,0.0,0
2580,2507 Leeper Dr,9,40.081463,-88.279758,15,4,0.000000,0.0,0
2581,2401-2415 Morrissey Park Dr,5,40.083264,-88.266737,16,4,0.000000,0.0,0
2582,2417-2427 Morrissey Park Dr,8,40.082793,-88.266738,16,4,0.000000,0.0,0


In [147]:
block_data = extranewone.copy()
block_data.head()

,address,safestay_score,latitude,longitude,block,num_streetlights,avg_wattage,num_crashes,has_police_station
0,2009 Moreland Blvd,5,40.140570,-88.253256,77,25,31.760000,6.0,1
1,2011 Moreland Blvd,6,40.140652,-88.252074,77,26,30.538462,6.0,1
2,2007 Moreland Blvd,7,40.140169,-88.252625,77,27,33.111111,6.0,1
3,2003 Moreland Blvd,8,40.139696,-88.252056,77,29,48.068966,6.0,1
4,2005 Moreland Blvd,0,40.139423,-88.253174,67,23,34.608696,9.0,1


In [148]:
block_data['streetlight_score'] = (block_data['num_streetlights'] - block_data['num_streetlights'].min()) / (block_data['num_streetlights'].max() - block_data['num_streetlights'].min())

block_data['wattage_score'] = (block_data['avg_wattage'] - block_data['avg_wattage'].min()) / (block_data['avg_wattage'].max() - block_data['avg_wattage'].min())

block_data['crash_score'] = (block_data['num_crashes'].max() - block_data['num_crashes']) / (block_data['num_crashes'].max() - block_data['num_crashes'].min()) 

# Experiment with larger weights for better granularity
block_data['safety_score'] = ((block_data['streetlight_score'] * 60) +   
                              (block_data['wattage_score'] * 40) +       
                              (block_data['has_police_station'] * 40) + 
                              (block_data['crash_score'] * 20))         

# Scale to 0–100 for a wider range
block_data['safety_score'] = (block_data['safety_score'] / block_data['safety_score'].max()) * 100

# Optionally scale down to 0–10
block_data['safety_score'] = (block_data['safety_score'] / 10).round()

In [149]:
block_data['safety_score'].unique()

array([ 6.,  7.,  8.,  4.,  5.,  2.,  9.,  3., 10.])

In [153]:
#change the type of safety score to int
block_data['safety_score'] = block_data['safety_score'].astype(int)
block_data.head()
#delete columns safestay_score, num_streetlights_avg_wattage, num_crashes_has_police_station, streetlight_score, wattage_score, crash_score
block_data2 = block_data.drop(columns=['safestay_score', 'num_streetlights', 'avg_wattage', 'num_crashes', 'has_police_station', 'streetlight_score', 'wattage_score', 'crash_score'])

block_data3 = block_data2.rename(columns={"safety_score": "safestay_score"})

#rearrange column ordering 
block_data3 = block_data3[['address', 'safestay_score', 'latitude', 'longitude', 'block']]
block_data3.head()

,address,safestay_score,latitude,longitude,block
0,2009 Moreland Blvd,6,40.140570,-88.253256,77
1,2011 Moreland Blvd,6,40.140652,-88.252074,77
2,2007 Moreland Blvd,6,40.140169,-88.252625,77
3,2003 Moreland Blvd,7,40.139696,-88.252056,77
4,2005 Moreland Blvd,6,40.139423,-88.253174,67


In [154]:
block_data3["safestay_score"].value_counts()

safestay_score
2     538
4     418
5     416
6     401
9     344
8     227
7     111
10     65
3      64
Name: count, dtype: int64

In [155]:
block_data3.head()

,address,safestay_score,latitude,longitude,block
0,2009 Moreland Blvd,6,40.140570,-88.253256,77
1,2011 Moreland Blvd,6,40.140652,-88.252074,77
2,2007 Moreland Blvd,6,40.140169,-88.252625,77
3,2003 Moreland Blvd,7,40.139696,-88.252056,77
4,2005 Moreland Blvd,6,40.139423,-88.253174,67


In [156]:
#export to csv 
block_data3.to_csv("/Users/stevenqie/Downloads/final_apartments_with_safestay_score.csv", index=False)
